In [300]:
# Подключаем библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tabula as tb
from scipy import stats
plt.rcParams['figure.figsize']=(10,5)

In [301]:
# Считаем все таблицы 
all_tabs_array = tb.read_pdf("tasks/Практика 22 ноября 2021.pdf", pages="all") 

Got stderr: Dec 21, 2021 7:28:49 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Dec 21, 2021 7:28:49 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Dec 21, 2021 7:28:50 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Dec 21, 2021 7:28:50 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Dec 21, 2021 7:28:50 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Dec 21, 2021 7:28:50 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Dec 21, 2021 7:28:50 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [302]:
len(all_tabs_array)

9

# ЗАДАЧА_1
Есть распространённое мнение, что дочери обычно больше похожи
на отцов, а сыновья – на матерей. Для проверки этого утверждения провели
исследование и составили следующую таблицу сопряженности:

In [303]:
# Уберем из нашей таблицы все лишнее и преобразуем нужные типы данных
# N0 - никакой связи нет
# N1 - связь есть
task_1 = pd.DataFrame(all_tabs_array[0])

In [304]:
task_1 = task_1.dropna(how="any")

In [305]:
task_1 = task_1.rename(columns={"Unnamed: 0": "gender","Ребенок больше похож на:": "alike"})

In [306]:
task_1 = task_1.join(task_1["alike"].str.split(expand=True)).drop("alike", axis=1)

In [307]:
task_1.columns = ["gender", "father", "mother"]

In [308]:
task_1[["father", "mother"]] = task_1[["father", "mother"]].applymap(int)

In [309]:
task_1.set_index(task_1["gender"]).drop("gender", axis=1)

,father,mother
gender,,
Мальчик,25,22
Девочка,19,17


In [310]:
chi2, p, dof, expected = stats.chi2_contingency(task_1[["father", "mother"]])

In [311]:
print(f"Chi-Squared: {chi2}\nDoF: {dof}\nP-value: {p}")

Chi-Squared: 0.0
DoF: 1
P-value: 1.0


In [312]:
# Мы не наблюдали статистически значимых отличий от ожидаемых значений, по этой причине мы не можем 
# Отказаться он нулевой гипотезы

# ЗАДАЧА_2
Два вуза готовят специалистов сходных специальностей.
Руководители этих вузов решили выяснить, можно ли считать примерно
одинаковым качество подготовки, получаемой студентами в обоих
институтах. Критерием качества подготовки было решено считать количество
выпускников каждого института, работающих по специальности. Был
проведен опрос выпускников обоих вузов, по результатам которого составлена
следующая таблица сопряженности:
Можно ли считать качество подготовки в институтах одинаковым?

In [313]:
# Поскольку объем данных в некоторых таблицах небольшой можно просто использовать матрицу 2х2
# Нулевая гипотеза - разницы в качестве вузов нет
# Альтернативная гипотеза - разницы есть
task_2 = all_tabs_array[1]

In [314]:
task_2

,Unnamed: 0,Работаете ли вы по специальности:
0,Институт,NaN
1,NaN,Да Нет
2,А,46 7
3,В,34 15


In [315]:
task_2 = np.array([[46, 7], [34, 15]])

In [316]:
chi2, p, dof, expected = stats.chi2_contingency(task_2)


In [317]:
# Вероятность случайно получить такое или большее отклонение превышает альфу (в качестве альфы мы взяли 0.05)
# В зависимоти от размера альфы мы можем решить принимать или отклонять гипотезу о различии
# Однако, в данном случае мы вынуждены отклонить альтернативную гипотезу о различии качетсва подготовки
# Хотя разница есть
print(f"Chi-Squared: {chi2}\nDoF: {dof}\nP-value: {p}")

Chi-Squared: 3.5884285189204324
DoF: 1
P-value: 0.05818323920233101


# ЗАДАЧА_3
В процессе разработки программы дополнительного образования
исследовалась связь между уровнем образования и предпочитаемой
слушателями формой подачи материала. Данные были сведены в таблицу:
Можно ли утверждать, что сущесвует связь между переменными

In [318]:
task_3 = all_tabs_array[2]

In [319]:
task_3

,Unnamed: 0,Форма подачи материала:
0,NaN,"Концентрированно, Подробно, практико-"
1,Образование,"наукообразно, текстово, ориентировано,"
2,NaN,"занятия от 20 минут и видеоролики,"
3,NaN,более занятия до 7 минут
4,Среднее,12 34
5,Среднее,NaN
6,NaN,8 17
7,профессиональное,NaN
8,Высшее,23 14


In [320]:
task_3 = np.array([[12, 34],[8, 17],[23, 14]])

In [321]:
chi2, p, dof, expected = stats.chi2_contingency(task_3)

In [322]:
# Наблюдаем значимую разницу, p-value меньше альфы
print(f"Chi-Squared: {chi2}\nDoF: {dof}\nP-value: {p}")

Chi-Squared: 11.965977452602326
DoF: 2
P-value: 0.0025212796056864324


# ЗАДАЧА_4
Принято считать, что девушки более старательно учатся, чем
молодые люди (не врут :) ), и реже пропускают занятия. провели исследование дляпроверки наличия связи между полом и посещаемостью занятий. Полученные
данные внесли в таблицу:
Можно ли утверждать, что существует связь между рассматриваемыми
переменными?

In [323]:
task_4 = all_tabs_array[3]

In [324]:
task_4

,Unnamed: 0,Unnamed: 1,Как часто вы пропускаете занятия:
0,NaN,Пол,Практически
1,NaN,NaN,Редко Часто
2,NaN,NaN,никогда
3,Девушки,NaN,16 26 9
4,Юноши,NaN,22 35 14


In [325]:
task_4 = [[16, 26, 9],[22, 35, 14]]

In [326]:
chi2, p, dof, expected = stats.chi2_contingency(task_4)

In [327]:
# Значимой связи не наблюдается
print(f"Chi-Squared: {chi2}\nDoF: {dof}\nP-value: {p}")

Chi-Squared: 0.08581140903842771
DoF: 2
P-value: 0.958001721040942


# ЗАДАЧА_5
В рамках исследования сравнивался уровень жизни в Москве и
регионах. В частности, была поставлена задача выяснить, зависит ли
материальное положение человека от типа населенного пункта, в котором он
проживает. Была построена следующая таблица сопряженности:
Можно ли утверждать, что указанная зависимость имеет место?

In [328]:
task_5 = all_tabs_array[4]

In [329]:
task_5

,Unnamed: 0,Материальное положение:
0,Тип населенного пункта,NaN
1,NaN,Хорошее Среднее Плохое
2,Москва и Санкт-Петербург,14 20 6
3,Города с численностью населения,NaN
4,NaN,10 23 8
5,более 1 млн. человек,NaN
6,Города с численностью населения от,NaN
7,NaN,7 18 12
8,100 тыс. до 1 млн. человек,NaN
9,Города с численностью населения от,NaN


In [330]:
task_5 = task_5.iloc[[2, 4, 7, 10, 13]]["Материальное положение:"].str.split()\
    .apply(lambda x: list(map(int, x)))

In [331]:
task_5 = np.array(list(task_5))

In [332]:
chi2, p, dof, expected = stats.chi2_contingency(task_5)

In [333]:
print(f"Chi-Squared: {chi2}\nDoF: {dof}\nP-value: {p}")

Chi-Squared: 15.418725962957883
DoF: 8
P-value: 0.05149706831734659


# ЗАДАЧА_6
В развитых странах люди, имеющие хорошее образование,
получают более высокий доход, чем те, у кого низкий уровень образования. В
некоторой стране было проведено исследование с целью выяснить,
выполняется ли это условие. Данные были сведены в таблицу сопряженности.
Можно ли утверждать, что выполняется указанное выше условие?

In [334]:
task_6 = all_tabs_array[5]

In [335]:
task_6

,Unnamed: 0,Unnamed: 1,"Уровень дохода, у.е.",Unnamed: 2
0,Образование,NaN,NaN,NaN
1,NaN,Менее 1000,1001 - 2000 2001 - 3000,более 3000
2,Среднее,11,12 5,1
3,Среднее,NaN,NaN,NaN
4,NaN,10,13 5,2
5,профессиональное,NaN,NaN,NaN
6,Высшее,6,14 11,7


In [336]:
task_6 = np.array([[11, 12, 5, 1], [10, 13, 5, 2], [6, 14, 11, 7]])

In [337]:
chi2, p, dof, expected = stats.chi2_contingency(task_6)

In [338]:
print(f"Chi-Squared: {chi2}\nDoF: {dof}\nP-value: {p}")

Chi-Squared: 9.233328854605322
DoF: 6
P-value: 0.16087479470387223
